In [1]:
import os
import sys
import numpy as np
import Data_processing as Dp
import tensorflow as tf
from tensorflow import keras
import imageio
from pathlib import Path 

ImportError: No module named tensorflow

### 1.0 Read data

In [ ]:
# Video options
_INPUT_SIZE = 224
_NUM_FRAMES = 64


In [ ]:
record_base_path =  "C:\\Users\\laiho\\Desktop\\machine_learning_project-cos80028\\UCF\\UCF_final_testing"

In [ ]:
video_list, label = Dp.load_dataset(record_base_path, 4)

### 2.0 model

Architecture:

vedio list -> vedio record:

- method selection
  - feature extraction
  - feed all frames to model for prediction
     - save prediction result in list
     - return result probability list


In [7]:
model_single_selector = tf.keras.models.load_model('single_selector.h5')
def Frame_selection_process(vedio_record,label, method = "both"):
    
    raw_input, _ = Dp.data_load(vedio_record, label)
    single_selector_input = Dp.feature_extraction(raw_input)
    single_selector_input = tf.convert_to_tensor(single_selector_input) 
    result = model_single_selector.predict(single_selector_input)
    # 这里直接获取了每个frame的得分
    print(result)

    return 0


In [16]:
# model_single_selector = tf.keras.models.load_model(location+'/'+record)
# model_global_selector = tf.keras.models.load_model(location+'/'+record)

model_single_selector = tf.keras.models.load_model('single_selector.h5')
model_global_selector = tf.keras.models.load_model('global_selector.h5')


def Frame_selection_process(vedio_record,label, method = "both"):
    
    if method == "both":
        # 用两个 模型对同一frame的计算得分相乘，输出最后的得分
        # single 
        raw_input, _ = Dp.data_load(vedio_record, label)
        single_selector_input = Dp.feature_extraction(raw_input)
        single_selector_input = tf.convert_to_tensor(single_selector_input) 
        result_single = model_single_selector.predict(single_selector_input)
        
        
        # global
        global_selector_pair_input = Dp.random_pair_generation(single_selector_input)
        result_global = model_global_selector.predict(global_selector_pair_input)        
            # 接着就要取有效得分
        
        
        # single global 得分相乘，得到最终得分
        
        result_both = 0  # 暂时放在这里，之后改
        return result_both
        
    elif method == "single":
        
        raw_input, _ = Dp.data_load(vedio_record, label)
        single_selector_input = Dp.feature_extraction(raw_input)
        single_selector_input = tf.convert_to_tensor(single_selector_input) 
        result = model_single_selector.predict(single_selector_input)
        # 这里直接获取了每个frame的得分
        print(np.array(result).shape)
        print(result)
        # 需要把 1.64.1的格式转化为 64长度的list
        
        
        return result
        
    elif method == "global":
        raw_input, _ = Dp.data_load(vedio_record, label)
        
        single_selector_input = Dp.feature_extraction(raw_input)
        single_selector_input = tf.convert_to_tensor(single_selector_input) 
        global_selector_pair_input = Dp.random_pair_generation(single_selector_input)
        result = model_global_selector.predict(global_selector_pair_input)
        print(result)
        
        
        # 这里我没有得到有效的得分，论文也没有提，似乎要观察训练完的模型到底能给我们什么才能定义几分方式
        # 已经得到了，果然是1，64，10长度的结果，接下来我们要根据label的位置来取
        result_score = 0
        return result_score
    
    else:
        print("illegal selection of methods")
    
    
    # 计划中这里是把一个vedio里所有frame的得分打包成list回传，之后的重建vedio工作由下面的步骤完成
    return 0



### 3.0 generate feature and vedio creation

for each record, generate feature and passing feature to model

Architecture:

Vedio list -> vedio record:

- pass to selection framework
- receive result probability list
- create new vedio based on that

In [8]:
def Frame_selection(video_list,label):
    length = len(video_list)
    for i in range(length):
        Frame_selection_process([video_list[i]], label[i],"single")
        
        
        
Frame_selection(video_list, label)



(64, 1, 224, 224, 3)
[[[10.497139 ]]

 [[10.497315 ]]

 [[10.498597 ]]

 [[10.499408 ]]

 [[10.497954 ]]

 [[10.499045 ]]

 [[10.499567 ]]

 [[10.49982  ]]

 [[10.499211 ]]

 [[10.4997015]]

 [[10.496088 ]]

 [[10.49788  ]]

 [[10.499829 ]]

 [[10.50001  ]]

 [[10.498052 ]]

 [[10.499269 ]]

 [[10.49902  ]]

 [[10.497717 ]]

 [[10.498931 ]]

 [[10.498909 ]]

 [[10.499751 ]]

 [[10.4976015]]

 [[10.498704 ]]

 [[10.498616 ]]

 [[10.49747  ]]

 [[10.499766 ]]

 [[10.498305 ]]

 [[10.4983635]]

 [[10.499141 ]]

 [[10.499093 ]]

 [[10.497458 ]]

 [[10.499837 ]]

 [[10.498185 ]]

 [[10.497246 ]]

 [[10.498647 ]]

 [[10.499452 ]]

 [[10.496802 ]]

 [[10.499194 ]]

 [[10.498874 ]]

 [[10.499225 ]]

 [[10.498076 ]]

 [[10.497871 ]]

 [[10.499177 ]]

 [[10.497289 ]]

 [[10.497834 ]]

 [[10.4988575]]

 [[10.500024 ]]

 [[10.499164 ]]

 [[10.499082 ]]

 [[10.498179 ]]

 [[10.497643 ]]

 [[10.238229 ]]

 [[10.2381115]]

 [[10.23887  ]]

 [[10.239095 ]]

 [[10.498766 ]]

 [[10.499167 ]]

 [[10.4983

In [17]:
def Frame_selection(video_list,label):
    length = len(video_list)
    
    for i in range(length):
        # 第一步获取 frame selection process 回传的score list
        score_list = Frame_selection_process([video_list[i]], label[i],"global")
        
        # 第二部把 score list 取前20位
        idx = (score_list).argsort()[:20]
        print(idx)
        idx.sort()
        print(idx)
        # 第三步用这个list 进行vedio重建
        # vedio 需要重建一个folder structure
        # 检测  label 做folder name是否存在， 如果不存在则建立folder，如果存在忽略
        # 然后把新的vedio放置在对应folder下        
        Path("/New_dataset/" + str(label)),mkdir(parents=True, exist_ok=True)
        with imageio.get_writer('./New_dataset'+lebel[0]+'.avi', mode='I') as writer:
            for i in range(20):
                image = local_image[:,idx[i]].squeeze()
                writer.append_data(image)
            writer.close()
        
        
Frame_selection(video_list,label)
# 这里就有点奇怪，似乎是要一个成功训练完的多分类模型才能判断要怎么写计分的方程

(64, 1, 224, 224, 3)
(64, 1, 6000) prepared_pair
[[ 3.9383109   4.3490887  -2.1016822  -2.3417075  -2.9706104   0.80763906
   1.3846742  -2.0448194  -2.3375492  -2.3878796 ]
 [ 3.9382904   4.349044   -2.1016426  -2.3417213  -2.9706233   0.8076154
   1.3846779  -2.0448127  -2.3375266  -2.387882  ]
 [ 3.9383678   4.349152   -2.1017144  -2.3417592  -2.9706697   0.80763274
   1.3846939  -2.0448396  -2.3375795  -2.38792   ]
 [ 3.938316    4.349093   -2.1016936  -2.3417046  -2.9705946   0.80762374
   1.3846228  -2.044791   -2.3375201  -2.3878636 ]
 [ 3.9383857   4.3491898  -2.1017423  -2.3417501  -2.9706519   0.80760235
   1.3846163  -2.044783   -2.3375285  -2.3878586 ]
 [ 3.9382935   4.349074   -2.1016614  -2.3416924  -2.9705935   0.80758756
   1.3846027  -2.0447707  -2.337505   -2.3878362 ]
 [ 4.041093    4.465376   -2.1364872  -2.4019926  -3.043213    0.80528474
   1.4130111  -2.0815475  -2.3827252  -2.4361615 ]
 [ 3.9384463   4.349249   -2.101763   -2.3417664  -2.970676    0.8075994
   1

AttributeError: 'int' object has no attribute 'argsort'

In [9]:
# 这边是对结果的列表根据大小取最高可能性的20个，然后用下面的代码重建vedio 格式为avi

idx = (prob_rsult).argsort()[:20]
print(idx)
idx.sort()
print(idx)

NameError: name 'prob_rsult' is not defined

In [ ]:
with imageio.get_writer('./testing_video'+lebel[0]+'.avi', mode='I') as writer:
    for i in range(20):
        image = local_image[:,idx[i]].squeeze()
        writer.append_data(image)
    writer.close()